In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

2024-03-24 15:53:42.207855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_file = "enemyHead_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_file)

valid_file = "enemyHead_valid.tfrecord"
valid_dataset = tf.data.TFRecordDataset(valid_file)

In [8]:
# for raw_record in dataset.take(10):
#     print(repr(raw_record))
    

In [4]:
keys_to_features = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/object/class/text': tf.io.VarLenFeature(tf.string),
    'image/object/class/label': tf.io.VarLenFeature(tf.int64),
}

def _parse_function(proto):
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    # Decode the image
    image = tf.image.decode_jpeg(parsed_features['image/encoded'])
    image = tf.image.resize(image, [416, 416])
    image = image / 255.0  # Normalize pixel values
    
    # Convert sparse tensor to dense, handling multiple entries if necessary
    label = tf.sparse.to_dense(parsed_features['image/object/class/label'], default_value=-1)
    text = tf.sparse.to_dense(parsed_features['image/object/class/text'], default_value='')
    
    # Decode bytes to string for text
    text = tf.cast(text, tf.string)
    return image, label, text

def save_images(dataset):
    # Map the dataset to parse the features
    images = dataset.map(_parse_function)
    
    # Directory to save the extracted images
    output_dir = 'extracted_images'
    os.makedirs(output_dir, exist_ok=True)
                
    # Iterate over each example
    for i, (image_tensor, label_tensor, text_tensor) in enumerate(images):
        # Save the image
        image_path = os.path.join(output_dir, f'image_{i}.jpg')
        tf.io.write_file(image_path, tf.image.encode_jpeg(image_tensor))
        print(f'Saved {image_path}')
        print(f'Label: {label_tensor.numpy()}, Text: {text_tensor.numpy()}')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(416, 416, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_dataset = train_dataset.map(_parse_function).batch(32)
valid_dataset = valid_dataset.map(_parse_function).batch(32)

history = model.fit(train_dataset, epochs=10)

In [ ]:

train_accuracy = model.evaluate(train_dataset)
print("Training Accuracy:", train_accuracy)

valid_accuracy = model.evaluate(valid_dataset)
print("Validation Accuracy:", valid_accuracy)
